In [4]:
import torch
import numpy as np
from torch import nn
from utils.model import Spice_model
from torchinfo import summary
import pandas as pd
from data_files.dataloader import MedleyDBLoader
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from utils.model import Spice_model, Spice_Encoder, Spice_Decoder
from utils.model_types import Spice_Decoder_1Unpool, Spice_Decoder_1Unpool, Deconv_block
from utils.training_script import Trainer
from optims.loss import Huber_loss, Recons_loss, Conf_loss
from data_files.dataset import CQT_Dataset

2023-08-01 19:42:08.617772: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-01 19:42:08.656975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 19:42:09.343492: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
import onnx

# Load the ONNX model
model = onnx.load("check.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %input[FLOAT, batch_sizex128]
) initializers (
  %enc_block.fc1.weight[FLOAT, 48x1024]
  %enc_block.fc1.bias[FLOAT, 48]
  %enc_block.fc2.weight[FLOAT, 1x48]
  %enc_block.fc2.bias[FLOAT, 1]
  %enc_block.conf_head.weight[FLOAT, 1x1024]
  %enc_block.conf_head.bias[FLOAT, 1]
  %dec_block.deconv_block1.deconv.weight[FLOAT, 512x256x3]
  %dec_block.deconv_block1.batchNorm.weight[FLOAT, 256]
  %dec_block.deconv_block1.batchNorm.bias[FLOAT, 256]
  %dec_block.deconv_block2.deconv.weight[FLOAT, 256x256x3]
  %dec_block.deconv_block3.deconv.weight[FLOAT, 256x256x3]
  %dec_block.deconv_block4.deconv.weight[FLOAT, 256x128x3]
  %dec_block.deconv_block4.batchNorm.weight[FLOAT, 128]
  %dec_block.deconv_block4.batchNorm.bias[FLOAT, 128]
  %dec_block.deconv_block5.deconv.weight[FLOAT, 128x64x3]
  %dec_block.deconv_block5.batchNorm.weight[FLOAT, 64]
  %dec_block.deconv_block5.batchNorm.bias[FLOAT, 64]
  %dec_block.deconv_block6.deconv.weight[FLOAT, 64x32x3]
  %dec_block.deconv_block6.ba

In [16]:
import onnxruntime as ort

ort_session = ort.InferenceSession("check.onnx", providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

outputs = ort_session.run(
    None,
    {"actual_input_1": np.random.randn(1, 128).astype(np.float32)},
)
print(outputs[0])

EP Error /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 710: device-side assert triggered ; GPU=0 ; hostname=pop-os ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=241 ; expr=cudaDeviceSynchronize(); 

 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecu

2023-07-31 21:09:51.187206836 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:614 CreateExecutionProviderInstance] Failed to create TensorrtExecutionProvider. Please reference https://onnxruntime.ai/docs/execution-providers/TensorRT-ExecutionProvider.html#requirements to ensure all dependencies are met.


RuntimeError: /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:114 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; std::conditional_t<THRW, void, onnxruntime::common::Status> = void] CUDA failure 710: device-side assert triggered ; GPU=0 ; hostname=pop-os ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=241 ; expr=cudaDeviceSynchronize(); 



In [2]:
mod = Deconv_block()
ss = torch.jit.trace(mod, torch.rand(10, 64, 64))

TypeError: MaxUnpool1d.forward() missing 1 required positional argument: 'indices'

In [7]:
print(ss.code)


def forward(self,
    input_1D: Tensor) -> Tensor:
  relu = self.relu
  batchNorm = self.batchNorm
  deconv = self.deconv
  _0 = (batchNorm).forward((deconv).forward(input_1D, ), )
  return (relu).forward(_0, )



In [ ]:
print(torch.__version__)
torch.manual_seed(191009)

In [8]:
@torch.jit.script
def foo_sum(x,y):
    return x+ y

In [9]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
        

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return foo_sum(new_h, new_h)

In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell2(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell2, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

In [ ]:
my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))

In [11]:
my_cell = MyCell()
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print("tracd",traced_cell,"end")
traced_cell(x, h)

tracd MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
) end


tensor([[ 0.7430,  1.4186,  1.5269,  0.7439],
        [ 0.6736, -0.4245,  0.9019, -0.2510],
        [ 1.0948,  0.4671,  0.8218,  0.3636]], grad_fn=<AddBackward0>)

In [12]:
#print(traced_cell.graph)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tensor:
  linear = self.linear
  new_h = torch.tanh(torch.add((linear).forward(x, ), h))
  return __torch__.foo_sum(new_h, new_h, )



In [ ]:
my_cell = MyCell2(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

print(traced_cell.dg.code)
print(traced_cell.code)

In [ ]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell2(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)
print(scripted_cell(x, h))#

In [ ]:

class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.script(MyCell2(scripted_gate))
        self.unpool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
    
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.unpool(xs[i], h)
        return y, h

rnn_loop = torch.jit.script( MyRNNLoop())
print(rnn_loop.code)
print(rnn_loop.cell.code)

In [ ]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        temp=0
        for i in h:
            temp+=y
        return torch.relu(y)+temp

traced = torch.jit.script(WrapRNN())
print(traced.loop.cell.code)

In [2]:
class convBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batch = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.maxPool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1, return_indices=True)
    def forward(self, input_1D):
        input_1D = self.conv(input_1D)
        input_1D = self.relu(self.batch(input_1D))
        input_1D, indx_mat = self.maxPool(input_1D)

        return input_1D, indx_mat

In [3]:
class deconvBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.deconv = nn.ConvTranspose1d(64, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchNorm = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.unpool = nn.MaxUnpool1d(kernel_size=3, stride=1, padding=0)
    
    def forward(self, input_1D1, idmat):
        input_1D = self.unpool(input_1D1, idmat)
        input_1D = self.deconv(input_1D)
        # batch norm
        input_1D = self.batchNorm(input_1D)
        # relu 
        input_1D = self.relu(input_1D)
        return input_1D

In [4]:
class modelFull(nn.Module):
    def __init__(self):
        super().__init__()
        self.bll = convBlock()
        self.deconv = deconvBlock()
    def forward(self, x):
            xx, y = self.bll(x)
            xz = self.deconv(xx, y)
            return xz

In [14]:
torch.jit.trace(Spice_model2(), torch.rand(1,  128))

Spice_model2(
  original_name=Spice_model2
  (enc_block): Spice_Encoder(
    original_name=Spice_Encoder
    (conv_block1): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block2): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block3): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNorm1d(original_name=BatchNorm1d)
      (relu): ReLU(original_name=ReLU)
      (maxPool): MaxPool1d(original_name=MaxPool1d)
    )
    (conv_block4): Conv_block(
      original_name=Conv_block
      (conv): Conv1d(original_name=Conv1d)
      (batch): BatchNo

In [6]:
s = modelFull()
summary(s, input_size=[2, 1, 128])

Layer (type:depth-idx)                   Output Shape              Param #
modelFull                                [2, 32, 66]               --
├─convBlock: 1-1                         [2, 64, 64]               --
│    └─Conv1d: 2-1                       [2, 64, 128]              192
│    └─BatchNorm1d: 2-2                  [2, 64, 128]              128
│    └─ReLU: 2-3                         [2, 64, 128]              --
│    └─MaxPool1d: 2-4                    [2, 64, 64]               --
├─deconvBlock: 1-2                       [2, 32, 66]               --
│    └─MaxUnpool1d: 2-5                  [2, 64, 66]               --
│    └─ConvTranspose1d: 2-6              [2, 32, 66]               6,144
│    └─BatchNorm1d: 2-7                  [2, 32, 66]               64
│    └─ReLU: 2-8                         [2, 32, 66]               --
Total params: 6,528
Trainable params: 6,528
Non-trainable params: 0
Total mult-adds (M): 0.86
Input size (MB): 0.00
Forward/backward pass size (MB): 0

In [ ]:
torch.jit.script(modelFull())

In [ ]:
spice = Spice_Decoder()

In [ ]:
torch.jit.script(spice)

In [ ]:
fs = 16000
med = MedleyDBLoader(fs )

In [28]:
f1 = 1110.0644999999954 
f2 = 5.684341886080802e-14
dive = f1/f2
dive

1.9528461205301544e+16

In [29]:
np.log2(dive)

54.11642779108129

In [55]:
data_pd = pd.read_pickle('./CQT_data/MIR1kfull2.pkl')

In [ ]:
nnp = np.load('./CQT_data/MedleyDB.npy')

In [56]:
fmax = np.max(data_pd.iloc[:, -1])
fmin = np.min(data_pd.iloc[:, -1]) 

In [57]:
fmin

20.0000488496063

In [58]:
data_pd.iloc[:15, -1]

4     147.070550
5     144.144107
6     146.255372
7     150.942966
8     148.706805
9     143.909850
10    134.453924
11    130.507561
12    129.241802
13    130.081145
14    131.580764
15    132.231130
16    132.231130
17    131.801133
18    127.603321
Name: 190, dtype: float64

In [32]:
data_pd.iloc[5, -1]

143.90985009261954

In [54]:
data_pd.loc[data_pd.iloc[:, -1]<20]

,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,190
344,0.005217,0.001848,0.005943,0.004031,0.005480,0.014429,0.005135,0.004574,0.006104,0.011626,...,0.013965,0.008797,0.006434,0.012345,0.009101,0.006797,0.003267,0.002287,0.000724,2.842171e-14
670,0.049793,0.055183,0.059535,0.067379,0.069461,0.072995,0.081257,0.084703,0.088940,0.089392,...,0.000111,0.000414,0.000422,0.000497,0.000529,0.000217,0.000247,0.000072,0.000217,1.250555e-11
3349,0.017742,0.000813,0.017584,0.017722,0.007697,0.003557,0.016785,0.013794,0.017591,0.016116,...,0.020716,0.019512,0.026083,0.033203,0.035206,0.015362,0.012048,0.013085,0.007575,2.842171e-14
7113,0.002018,0.038668,0.054554,0.031007,0.028359,0.011903,0.013359,0.050509,0.047573,0.019750,...,0.000478,0.000263,0.000362,0.000425,0.000416,0.000314,0.000255,0.000276,0.000321,2.842171e-14
7965,0.037131,0.035321,0.031628,0.027804,0.035301,0.043344,0.032813,0.031208,0.029053,0.027769,...,0.005665,0.007035,0.004666,0.006148,0.003617,0.000516,0.000283,0.000170,0.000498,1.893487e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240573,0.005941,0.004063,0.004148,0.006147,0.005780,0.003504,0.002276,0.002994,0.004917,0.005770,...,0.000430,0.000277,0.000088,0.000139,0.000028,0.000089,0.000079,0.000065,0.000072,9.123369e-12
249465,0.007889,0.006155,0.011544,0.014008,0.002876,0.009931,0.015477,0.008524,0.009942,0.010375,...,0.000177,0.000099,0.000161,0.000101,0.000135,0.000128,0.000038,0.000027,0.000044,1.776357e-11
250009,0.017629,0.026075,0.028820,0.017638,0.003201,0.016272,0.011814,0.003255,0.014340,0.034213,...,0.000214,0.000032,0.000142,0.000151,0.000130,0.000080,0.000051,0.000044,0.000054,2.060574e-11
250766,0.089455,0.081540,0.073750,0.095536,0.104619,0.096023,0.114201,0.116220,0.100998,0.105345,...,0.000017,0.000022,0.000046,0.000074,0.000073,0.000109,0.000094,0.000068,0.000100,1.828575e+01


In [ ]:
#data_pd = pd.DataFrame(data=nnp) 
train, val = train_test_split(data_pd, train_size=0.8, test_size=0.2, random_state=1)
print("train shape: ", train.shape)
train_batches = DataLoader(CQT_Dataset(data=train, mode='train'), batch_size=64, shuffle=True)
print("train_batch NUmber: ", len(train_batches))
diff, slice1, slice2, f0 = next(iter(train_batches))
print(f"diff batch shape: {diff.size()}")
print(f"slice1 batch shape: {slice1.size()}")
print(f"slice2 batch shape: {f0.size()}")

spice = Spice_model()


for b in train_batches:
    print("batch shape:", b[0].shape)
    pitch_diff, x_1, x_2, f0 = b
    x_1 = x_1.type(torch.FloatTensor)
    #print(x_1.shape, x_1.type())
    a, x, y = spice(x_1)
    print(a.size(), x.size(), y.size())

In [ ]:
xx=torch.arange(0, 25, 1)

In [ ]:
xx = xx.reshape((5,-1))

In [ ]:
xx.size()

In [123]:
df = pd.read_pickle('./CQT_data/MDBSynth.pkl')
df.shape

(358919, 191)

In [124]:
# remove rows of cqt where label (last) column is zero
df.drop(df.loc[df.iloc[:, -1]<=0].index, inplace=True)
df.shape

(181432, 191)

In [125]:
df = df.sample(frac=1, random_state=10).reset_index()
df.shape

(181432, 192)

In [126]:
df.drop(columns=['index'],axis=1, inplace=True)
df.shape

(181432, 191)

In [106]:
fmax = np.max(df.iloc[:, -1])
fmin = np.min(df.iloc[:, -1]) 
print(fmax, fmin)

1205.3295446000004 -5.684341886080802e-14


In [104]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,181,182,183,184,185,186,187,188,189,190
0,0.093583,0.103289,0.108589,0.114461,0.126163,0.141803,0.163479,0.186689,0.207631,0.221017,...,0.003004,0.001498,0.002785,0.003538,0.002594,0.003086,0.003188,0.001229,0.000061,66.010860
1,0.000082,0.000233,0.000090,0.000190,0.000066,0.000029,0.000197,0.000035,0.000137,0.000152,...,0.008210,0.008015,0.005654,0.004071,0.002212,0.002080,0.001428,0.000503,0.000096,309.451418
2,0.000258,0.000223,0.000260,0.000323,0.000325,0.000267,0.000267,0.000327,0.000348,0.000338,...,0.003706,0.005361,0.010392,0.009005,0.004797,0.001981,0.000298,0.000261,0.000129,164.620700
3,0.000109,0.000086,0.000039,0.000051,0.000037,0.000029,0.000061,0.000103,0.000143,0.000165,...,0.000322,0.000238,0.000251,0.000221,0.000133,0.000084,0.000069,0.000024,0.000017,299.930182
4,0.000092,0.000159,0.000204,0.000082,0.000007,0.000048,0.000038,0.000063,0.000103,0.000153,...,0.000320,0.000105,0.000541,0.001700,0.001701,0.001012,0.000401,0.000046,0.000040,123.772538


In [ ]:
np.arange(0,25,1).reshape(5,5)[:,-1:]

In [49]:
a = np.arange(0,15,1).reshape(5,3)
a = np.hstack((a, np.array([-0.25,0.7,0,-0.5,0]).reshape(-1,1)))
a

array([[ 0.  ,  1.  ,  2.  , -0.25],
       [ 3.  ,  4.  ,  5.  ,  0.7 ],
       [ 6.  ,  7.  ,  8.  ,  0.  ],
       [ 9.  , 10.  , 11.  , -0.5 ],
       [12.  , 13.  , 14.  ,  0.  ]])

In [59]:
df = pd.DataFrame(a)
df.shape

(5, 4)

In [68]:
df.drop('index',axis=1, inplace=True)

In [69]:
df.head()

,0,1,2,3
0,6.0,7.0,8.0,0.00
1,9.0,10.0,11.0,-0.50
2,0.0,1.0,2.0,-0.25
3,12.0,13.0,14.0,0.00
4,3.0,4.0,5.0,0.70


In [ ]:
np.min(df.iloc[:, -1])

In [79]:
a = np.array([1,2,0,0,3,4,0,0,5,4,6,0,2,1])

In [82]:
a = a[a!=0]

In [83]:
a

array([1, 2, 3, 4, 5, 4, 6, 2, 1])

In [3]:
torch.jit.script(Spice_model2())

RuntimeError: isIntList() INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/core/ivalue_inl.h":1938, please report a bug to PyTorch. Expected IntList but got Int

In [2]:
s = Spice_model2()
summary(s, input_size=[10, 128])

Layer (type:depth-idx)                   Output Shape              Param #
Spice_model2                             [10, 1]                   --
├─Spice_Encoder: 1-1                     [10, 1]                   --
│    └─Conv_block: 2-1                   [10, 64, 64]              --
│    │    └─Conv1d: 3-1                  [10, 64, 128]             192
│    │    └─BatchNorm1d: 3-2             [10, 64, 128]             128
│    │    └─ReLU: 3-3                    [10, 64, 128]             --
│    │    └─MaxPool1d: 3-4               [10, 64, 64]              --
│    └─Conv_block: 2-2                   [10, 128, 32]             --
│    │    └─Conv1d: 3-5                  [10, 128, 64]             24,576
│    │    └─BatchNorm1d: 3-6             [10, 128, 64]             256
│    │    └─ReLU: 3-7                    [10, 128, 64]             --
│    │    └─MaxPool1d: 3-8               [10, 128, 32]             --
│    └─Conv_block: 2-3                   [10, 256, 16]             --
│    │  

In [ ]:
for n, param in s.named_parameters():
    print(type(param), param.requires_grad)

In [85]:
def scaling_factor(Q):
    # take care of negative inside log
    # for Mir1k values are 666.6649397699019, 66.9456331525636
    # since we are not using interpolated labels for training we use original 
    # dataset values for fmin and fmax
    # Mir1k
    fmax, fmin = 666.664939769901, 66.9456331525636256
    # MDBSynth
    print('inside Q', fmax, fmin)
    return 1 / (Q * np.log2(fmax / fmin))

In [86]:
scaling_factor(24)

inside Q 666.664939769901 66.94563315256363


0.012565718708847283